In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pandas as pd
df = pd.read_csv('Final_Dataset.txt', index_col=0)

In [ ]:
#To perfrom comparable models
df['rating'] = df['duration_seconds']*1.0/df['max_duration']

In [ ]:
name_to_idx = {mid: idx for idx, mid in enumerate(df['original_name'].unique())}
df['movie_id'] = df['original_name'].map(name_to_idx)

In [1]:
# convert the movie_id and user_id_maped to string for the vectors
df['movie_id'] = df['movie_id'].astype(str)
df['user_id_maped'] = df['user_id_maped'].astype(str)

df = df[['movie_id','program_class', 'program_genre', 'hd', 'isPopular', 'user_id_maped','isActive', 'preferred_program_genre', 'rating']]

In [ ]:
train = df.sample(frac=0.8,random_state=5)
test = df.drop(train.index)
test = test.loc[test.movie_id.isin(train.movie_id.unique())]
test = test.loc[test.user_id_maped.isin(train.user_id_maped.unique())]
test.reset_index(drop=True, inplace=True)

# save
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)
df.to_csv('df.csv', index= False)

In [ ]:
user_df = train[['user_id_maped','isActive', 'preferred_program_genre']]
movie_df = train[['movie_id','program_class', 'program_genre', 'hd', 'isPopular']]

test_user_df = test[['user_id_maped','isActive', 'preferred_program_genre']]
test_movie_df = test[['movie_id','program_class', 'program_genre', 'hd', 'isPopular']]

ohe_user = OneHotEncoder(handle_unknown='ignore',drop='first', min_frequency=5)
user_df = ohe_user.fit_transform(user_df)
test_user_df = ohe_user.transform(test_user_df)

ohe_movie = OneHotEncoder(handle_unknown='ignore',drop='first')
movie_df = ohe_movie.fit_transform(movie_df)
test_movie_df = ohe_movie.transform(test_movie_df)

In [ ]:
user_df.shape

(571760, 15370)

In [ ]:
movie_df.shape

(571760, 1806)

In [ ]:
#free memory
del df
del user_counts
del user_genres

In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

# Define vectors
user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

# Concatenate
concat = keras.layers.Concatenate()([user_input, item_input])

#hidden layers
hidden = keras.layers.Dense(32, activation='relu')(concat)
# hidden = keras.layers.Dense(32, activation='relu')(hidden)

# Output layer
output = keras.layers.Dense(1)(hidden)

#model
model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

# training
model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

Epoch 1/5


2023-05-23 14:40:49.398796: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 74s 4ms/step - loss: 0.1084 - RMSE: 0.3293 - MAE: 0.2338 - val_loss: 0.1043 - val_RMSE: 0.3229 - val_MAE: 0.2308
Epoch 2/5
17868/17868 [==============================] - 67s 4ms/step - loss: 0.0994 - RMSE: 0.3152 - MAE: 0.2190 - val_loss: 0.1030 - val_RMSE: 0.3210 - val_MAE: 0.2231
Epoch 3/5
17868/17868 [==============================] - 66s 4ms/step - loss: 0.0958 - RMSE: 0.3095 - MAE: 0.2130 - val_loss: 0.1029 - val_RMSE: 0.3207 - val_MAE: 0.2209
Epoch 4/5
17868/17868 [==============================] - 66s 4ms/step - loss: 0.0931 - RMSE: 0.3052 - MAE: 0.2083 - val_loss: 0.1037 - val_RMSE: 0.3220 - val_MAE: 0.2198
Epoch 5/5
17868/17868 [==============================] - 67s 4ms/step - loss: 0.0910 - RMSE: 0.3017 - MAE: 0.2044 - val_loss: 0.1034 - val_RMSE: 0.3216 - val_MAE: 0.2187


In [ ]:
results_relu = pd.DataFrame(model.history.history)
results_relu

,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108435,0.329295,0.233805,0.104267,0.322904,0.230808
1,0.099352,0.315201,0.218979,0.103017,0.320962,0.223107
2,0.095761,0.309452,0.213050,0.102867,0.320729,0.220939
3,0.093124,0.305162,0.208283,0.103702,0.322028,0.219838
4,0.091000,0.301662,0.204395,0.103409,0.321573,0.218729


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

hidden = keras.layers.Dense(32, activation='tanh')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_tanh = pd.DataFrame(model.history.history)
results_tanh

Epoch 1/5
17868/17868 [==============================] - 72s 4ms/step - loss: 0.1129 - RMSE: 0.3360 - MAE: 0.2479 - val_loss: 0.1106 - val_RMSE: 0.3326 - val_MAE: 0.2455
Epoch 2/5
17868/17868 [==============================] - 66s 4ms/step - loss: 0.1076 - RMSE: 0.3280 - MAE: 0.2375 - val_loss: 0.1086 - val_RMSE: 0.3295 - val_MAE: 0.2376
Epoch 3/5
17868/17868 [==============================] - 67s 4ms/step - loss: 0.1054 - RMSE: 0.3247 - MAE: 0.2312 - val_loss: 0.1069 - val_RMSE: 0.3270 - val_MAE: 0.2336
Epoch 4/5
17868/17868 [==============================] - 67s 4ms/step - loss: 0.1028 - RMSE: 0.3206 - MAE: 0.2252 - val_loss: 0.1051 - val_RMSE: 0.3241 - val_MAE: 0.2275
Epoch 5/5
17868/17868 [==============================] - 67s 4ms/step - loss: 0.1006 - RMSE: 0.3171 - MAE: 0.2201 - val_loss: 0.1043 - val_RMSE: 0.3230 - val_MAE: 0.2267


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.112926,0.336044,0.247901,0.110606,0.332576,0.245539
1,0.107566,0.327972,0.237469,0.108601,0.329547,0.237641
2,0.105446,0.324724,0.231151,0.106913,0.326976,0.233639
3,0.102767,0.320572,0.225232,0.105065,0.324138,0.227463
4,0.100579,0.317142,0.220125,0.104299,0.322953,0.226739


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

hidden = keras.layers.Dense(32, activation='sigmoid')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_sigmoid = pd.DataFrame(model.history.history)
results_sigmoid

Epoch 1/5


2023-05-23 15:17:16.252898: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 74s 4ms/step - loss: 0.1133 - RMSE: 0.3366 - MAE: 0.2483 - val_loss: 0.1100 - val_RMSE: 0.3316 - val_MAE: 0.2440
Epoch 2/5
17868/17868 [==============================] - 66s 4ms/step - loss: 0.1066 - RMSE: 0.3265 - MAE: 0.2360 - val_loss: 0.1064 - val_RMSE: 0.3262 - val_MAE: 0.2365
Epoch 3/5
17868/17868 [==============================] - 66s 4ms/step - loss: 0.1034 - RMSE: 0.3216 - MAE: 0.2298 - val_loss: 0.1052 - val_RMSE: 0.3243 - val_MAE: 0.2363
Epoch 4/5
17868/17868 [==============================] - 66s 4ms/step - loss: 0.1014 - RMSE: 0.3185 - MAE: 0.2253 - val_loss: 0.1047 - val_RMSE: 0.3235 - val_MAE: 0.2388
Epoch 5/5
17868/17868 [==============================] - 65s 4ms/step - loss: 0.0996 - RMSE: 0.3157 - MAE: 0.2211 - val_loss: 0.1035 - val_RMSE: 0.3217 - val_MAE: 0.2264


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.113293,0.336590,0.248254,0.109968,0.331614,0.244031
1,0.106606,0.326505,0.235995,0.106401,0.326192,0.236504
2,0.103443,0.321626,0.229794,0.105191,0.324331,0.236277
3,0.101423,0.318470,0.225345,0.104653,0.323500,0.238833
4,0.099646,0.315668,0.221112,0.103469,0.321666,0.226358


In [ ]:
#Test Neurons

#Test Neurons

#Test Neurons

#Test Neurons

#Test Neurons

#Test Neurons

In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#4 neurons
hidden = keras.layers.Dense(4, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_neurons_4 = pd.DataFrame(model.history.history)
results_neurons_4

Epoch 1/5


2023-05-23 16:06:22.498636: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 39s 2ms/step - loss: 0.1112 - RMSE: 0.3335 - MAE: 0.2419 - val_loss: 0.1061 - val_RMSE: 0.3258 - val_MAE: 0.2341
Epoch 2/5
17868/17868 [==============================] - 32s 2ms/step - loss: 0.1027 - RMSE: 0.3205 - MAE: 0.2269 - val_loss: 0.1051 - val_RMSE: 0.3242 - val_MAE: 0.2294
Epoch 3/5
17868/17868 [==============================] - 32s 2ms/step - loss: 0.1006 - RMSE: 0.3172 - MAE: 0.2238 - val_loss: 0.1045 - val_RMSE: 0.3233 - val_MAE: 0.2320
Epoch 4/5
17868/17868 [==============================] - 31s 2ms/step - loss: 0.0995 - RMSE: 0.3154 - MAE: 0.2218 - val_loss: 0.1044 - val_RMSE: 0.3231 - val_MAE: 0.2269
Epoch 5/5
17868/17868 [==============================] - 31s 2ms/step - loss: 0.0987 - RMSE: 0.3141 - MAE: 0.2205 - val_loss: 0.1044 - val_RMSE: 0.3232 - val_MAE: 0.2242


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.111207,0.333477,0.241875,0.106124,0.325767,0.234128
1,0.102722,0.320503,0.226874,0.105113,0.324211,0.229443
2,0.100635,0.317230,0.223753,0.104542,0.323330,0.232030
3,0.099471,0.315390,0.221821,0.104381,0.323080,0.226919
4,0.098669,0.314116,0.220528,0.104437,0.323167,0.224157


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#8 neurons
hidden = keras.layers.Dense(8, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_neurons_8 = pd.DataFrame(model.history.history)
results_neurons_8

Epoch 1/5
17868/17868 [==============================] - 45s 2ms/step - loss: 0.1095 - RMSE: 0.3308 - MAE: 0.2355 - val_loss: 0.1049 - val_RMSE: 0.3239 - val_MAE: 0.2252
Epoch 2/5
17868/17868 [==============================] - 38s 2ms/step - loss: 0.1009 - RMSE: 0.3176 - MAE: 0.2204 - val_loss: 0.1039 - val_RMSE: 0.3223 - val_MAE: 0.2250
Epoch 3/5
17868/17868 [==============================] - 39s 2ms/step - loss: 0.0980 - RMSE: 0.3131 - MAE: 0.2163 - val_loss: 0.1037 - val_RMSE: 0.3220 - val_MAE: 0.2230
Epoch 4/5
17868/17868 [==============================] - 39s 2ms/step - loss: 0.0964 - RMSE: 0.3105 - MAE: 0.2136 - val_loss: 0.1036 - val_RMSE: 0.3218 - val_MAE: 0.2224
Epoch 5/5
17868/17868 [==============================] - 41s 2ms/step - loss: 0.0952 - RMSE: 0.3086 - MAE: 0.2115 - val_loss: 0.1037 - val_RMSE: 0.3220 - val_MAE: 0.2227


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.109451,0.330834,0.235481,0.104916,0.323907,0.225161
1,0.100850,0.317569,0.220395,0.103894,0.322326,0.224995
2,0.098040,0.313114,0.216263,0.103677,0.321989,0.222994
3,0.096419,0.310513,0.213640,0.103572,0.321826,0.222364
4,0.095218,0.308575,0.211527,0.103656,0.321956,0.222731


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#16 neurons
hidden = keras.layers.Dense(16, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_neurons_16 = pd.DataFrame(model.history.history)
results_neurons_16

Epoch 1/5


2023-05-23 16:14:57.200808: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 53s 3ms/step - loss: 0.1083 - RMSE: 0.3292 - MAE: 0.2337 - val_loss: 0.1044 - val_RMSE: 0.3231 - val_MAE: 0.2309
Epoch 2/5
17868/17868 [==============================] - 47s 3ms/step - loss: 0.0997 - RMSE: 0.3158 - MAE: 0.2193 - val_loss: 0.1033 - val_RMSE: 0.3213 - val_MAE: 0.2287
Epoch 3/5
17868/17868 [==============================] - 47s 3ms/step - loss: 0.0963 - RMSE: 0.3104 - MAE: 0.2141 - val_loss: 0.1029 - val_RMSE: 0.3207 - val_MAE: 0.2232
Epoch 4/5
17868/17868 [==============================] - 48s 3ms/step - loss: 0.0941 - RMSE: 0.3067 - MAE: 0.2102 - val_loss: 0.1030 - val_RMSE: 0.3210 - val_MAE: 0.2181
Epoch 5/5
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0924 - RMSE: 0.3040 - MAE: 0.2072 - val_loss: 0.1029 - val_RMSE: 0.3208 - val_MAE: 0.2209


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108349,0.329164,0.233744,0.104367,0.323059,0.230863
1,0.099723,0.315789,0.219342,0.103260,0.321342,0.228709
2,0.096348,0.310399,0.214075,0.102862,0.320721,0.223157
3,0.094071,0.306711,0.210243,0.103049,0.321013,0.218133
4,0.092418,0.304004,0.207232,0.102940,0.320842,0.220898


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#32 neurons
hidden = keras.layers.Dense(32, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_neurons_32 = pd.DataFrame(model.history.history)
results_neurons_32

Epoch 1/5
17868/17868 [==============================] - 48s 3ms/step - loss: 0.1096 - RMSE: 0.3311 - MAE: 0.2372 - val_loss: 0.1050 - val_RMSE: 0.3240 - val_MAE: 0.2313
Epoch 2/5
17868/17868 [==============================] - 40s 2ms/step - loss: 0.1009 - RMSE: 0.3177 - MAE: 0.2232 - val_loss: 0.1035 - val_RMSE: 0.3218 - val_MAE: 0.2274
Epoch 3/5
17868/17868 [==============================] - 40s 2ms/step - loss: 0.0979 - RMSE: 0.3129 - MAE: 0.2181 - val_loss: 0.1036 - val_RMSE: 0.3219 - val_MAE: 0.2212
Epoch 4/5
17868/17868 [==============================] - 40s 2ms/step - loss: 0.0963 - RMSE: 0.3103 - MAE: 0.2151 - val_loss: 0.1033 - val_RMSE: 0.3214 - val_MAE: 0.2273
Epoch 5/5
17868/17868 [==============================] - 42s 2ms/step - loss: 0.0951 - RMSE: 0.3084 - MAE: 0.2130 - val_loss: 0.1034 - val_RMSE: 0.3216 - val_MAE: 0.2234


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.109630,0.331104,0.237243,0.104979,0.324005,0.231296
1,0.100919,0.317678,0.223156,0.103536,0.321770,0.227411
2,0.097921,0.312923,0.218118,0.103626,0.321910,0.221223
3,0.096259,0.310257,0.215115,0.103315,0.321426,0.227255
4,0.095095,0.308375,0.213000,0.103421,0.321592,0.223365


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#64 neurons
hidden = keras.layers.Dense(64, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_neurons_64 = pd.DataFrame(model.history.history)
results_neurons_64

Epoch 1/5


2023-05-23 16:29:10.995338: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 46s 3ms/step - loss: 0.1089 - RMSE: 0.3300 - MAE: 0.2339 - val_loss: 0.1049 - val_RMSE: 0.3239 - val_MAE: 0.2268
Epoch 2/5
17868/17868 [==============================] - 38s 2ms/step - loss: 0.1008 - RMSE: 0.3174 - MAE: 0.2187 - val_loss: 0.1036 - val_RMSE: 0.3219 - val_MAE: 0.2216
Epoch 3/5
17868/17868 [==============================] - 38s 2ms/step - loss: 0.0981 - RMSE: 0.3132 - MAE: 0.2143 - val_loss: 0.1035 - val_RMSE: 0.3217 - val_MAE: 0.2231
Epoch 4/5
17868/17868 [==============================] - 39s 2ms/step - loss: 0.0964 - RMSE: 0.3106 - MAE: 0.2116 - val_loss: 0.1035 - val_RMSE: 0.3218 - val_MAE: 0.2238
Epoch 5/5
17868/17868 [==============================] - 39s 2ms/step - loss: 0.0952 - RMSE: 0.3086 - MAE: 0.2095 - val_loss: 0.1037 - val_RMSE: 0.3219 - val_MAE: 0.2178


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108878,0.329966,0.233899,0.104907,0.323894,0.226780
1,0.100754,0.317418,0.218704,0.103632,0.321919,0.221642
2,0.098100,0.313210,0.214327,0.103504,0.321721,0.223114
3,0.096450,0.310563,0.211581,0.103528,0.321757,0.223823
4,0.095209,0.308560,0.209532,0.103650,0.321948,0.217758


In [ ]:
#Test Layers

#Test Layers

#Test Layers

#Test Layers

#Test Layers

#Test Layers

In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#single hidden layer
hidden = keras.layers.Dense(16, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_layers_1 = pd.DataFrame(model.history.history)
results_layers_1

Epoch 1/5
17868/17868 [==============================] - 56s 3ms/step - loss: 0.1091 - RMSE: 0.3303 - MAE: 0.2364 - val_loss: 0.1045 - val_RMSE: 0.3233 - val_MAE: 0.2308
Epoch 2/5
17868/17868 [==============================] - 49s 3ms/step - loss: 0.1000 - RMSE: 0.3162 - MAE: 0.2216 - val_loss: 0.1031 - val_RMSE: 0.3211 - val_MAE: 0.2203
Epoch 3/5
17868/17868 [==============================] - 49s 3ms/step - loss: 0.0966 - RMSE: 0.3109 - MAE: 0.2161 - val_loss: 0.1028 - val_RMSE: 0.3206 - val_MAE: 0.2259
Epoch 4/5
17868/17868 [==============================] - 50s 3ms/step - loss: 0.0944 - RMSE: 0.3072 - MAE: 0.2124 - val_loss: 0.1028 - val_RMSE: 0.3207 - val_MAE: 0.2224
Epoch 5/5
17868/17868 [==============================] - 50s 3ms/step - loss: 0.0927 - RMSE: 0.3045 - MAE: 0.2096 - val_loss: 0.1030 - val_RMSE: 0.3209 - val_MAE: 0.2254


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.109101,0.330304,0.236418,0.104491,0.323250,0.230817
1,0.099995,0.316220,0.221557,0.103136,0.321149,0.220267
2,0.096638,0.310866,0.216116,0.102803,0.320629,0.225893
3,0.094382,0.307216,0.212420,0.102830,0.320671,0.222444
4,0.092729,0.304514,0.209594,0.103000,0.320936,0.225403


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#2 layers
hidden = keras.layers.Dense(16, activation='relu')(concat)
hidden = keras.layers.Dense(16, activation='relu')(hidden)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_layers_2 = pd.DataFrame(model.history.history)
results_layers_2


Epoch 1/5


2023-05-23 16:50:52.528879: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 56s 3ms/step - loss: 0.1086 - RMSE: 0.3295 - MAE: 0.2341 - val_loss: 0.1044 - val_RMSE: 0.3231 - val_MAE: 0.2315
Epoch 2/5
17868/17868 [==============================] - 49s 3ms/step - loss: 0.0997 - RMSE: 0.3157 - MAE: 0.2169 - val_loss: 0.1034 - val_RMSE: 0.3215 - val_MAE: 0.2203
Epoch 3/5
17868/17868 [==============================] - 49s 3ms/step - loss: 0.0956 - RMSE: 0.3092 - MAE: 0.2093 - val_loss: 0.1026 - val_RMSE: 0.3203 - val_MAE: 0.2181
Epoch 4/5
17868/17868 [==============================] - 49s 3ms/step - loss: 0.0926 - RMSE: 0.3042 - MAE: 0.2034 - val_loss: 0.1032 - val_RMSE: 0.3212 - val_MAE: 0.2262
Epoch 5/5
17868/17868 [==============================] - 48s 3ms/step - loss: 0.0902 - RMSE: 0.3003 - MAE: 0.1991 - val_loss: 0.1034 - val_RMSE: 0.3216 - val_MAE: 0.2149


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108598,0.329542,0.234105,0.104387,0.323090,0.231466
1,0.099666,0.315700,0.216915,0.103360,0.321497,0.220299
2,0.095595,0.309185,0.209255,0.102583,0.320286,0.218075
3,0.092552,0.304224,0.203432,0.103166,0.321194,0.226216
4,0.090207,0.300345,0.199062,0.103424,0.321597,0.214900


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#3 layers
hidden = keras.layers.Dense(16, activation='relu')(concat)
hidden = keras.layers.Dense(16, activation='relu')(hidden)
hidden = keras.layers.Dense(16, activation='relu')(hidden)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_layers_3 = pd.DataFrame(model.history.history)
results_layers_3


Epoch 1/5
17868/17868 [==============================] - 56s 3ms/step - loss: 0.1082 - RMSE: 0.3290 - MAE: 0.2326 - val_loss: 0.1045 - val_RMSE: 0.3233 - val_MAE: 0.2209
Epoch 2/5
17868/17868 [==============================] - 49s 3ms/step - loss: 0.0994 - RMSE: 0.3152 - MAE: 0.2158 - val_loss: 0.1032 - val_RMSE: 0.3212 - val_MAE: 0.2164
Epoch 3/5
17868/17868 [==============================] - 50s 3ms/step - loss: 0.0954 - RMSE: 0.3089 - MAE: 0.2082 - val_loss: 0.1039 - val_RMSE: 0.3224 - val_MAE: 0.2232
Epoch 4/5
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0924 - RMSE: 0.3040 - MAE: 0.2022 - val_loss: 0.1036 - val_RMSE: 0.3219 - val_MAE: 0.2137
Epoch 5/5
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0899 - RMSE: 0.2998 - MAE: 0.1973 - val_loss: 0.1039 - val_RMSE: 0.3224 - val_MAE: 0.2173


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108243,0.329004,0.232611,0.104518,0.323292,0.220877
1,0.099372,0.315234,0.215805,0.103157,0.321180,0.216434
2,0.095418,0.308897,0.208178,0.103941,0.322399,0.223212
3,0.092391,0.303959,0.202239,0.103611,0.321886,0.213688
4,0.089908,0.299846,0.197309,0.103933,0.322386,0.217250


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#4 layers
hidden = keras.layers.Dense(16, activation='relu')(concat)
hidden = keras.layers.Dense(16, activation='relu')(hidden)
hidden = keras.layers.Dense(16, activation='relu')(hidden)
hidden = keras.layers.Dense(16, activation='relu')(hidden)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_layers_4 = pd.DataFrame(model.history.history)
results_layers_4

Epoch 1/5


2023-05-23 17:02:51.059051: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 53s 3ms/step - loss: 0.1086 - RMSE: 0.3295 - MAE: 0.2334 - val_loss: 0.1047 - val_RMSE: 0.3236 - val_MAE: 0.2359
Epoch 2/5
17868/17868 [==============================] - 48s 3ms/step - loss: 0.0993 - RMSE: 0.3151 - MAE: 0.2156 - val_loss: 0.1030 - val_RMSE: 0.3210 - val_MAE: 0.2174
Epoch 3/5
17868/17868 [==============================] - 50s 3ms/step - loss: 0.0951 - RMSE: 0.3083 - MAE: 0.2074 - val_loss: 0.1031 - val_RMSE: 0.3210 - val_MAE: 0.2130
Epoch 4/5
17868/17868 [==============================] - 52s 3ms/step - loss: 0.0917 - RMSE: 0.3029 - MAE: 0.2009 - val_loss: 0.1037 - val_RMSE: 0.3220 - val_MAE: 0.2131
Epoch 5/5
17868/17868 [==============================] - 48s 3ms/step - loss: 0.0888 - RMSE: 0.2980 - MAE: 0.1952 - val_loss: 0.1044 - val_RMSE: 0.3231 - val_MAE: 0.2181


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108583,0.329519,0.233421,0.104697,0.323570,0.235927
1,0.099285,0.315095,0.215633,0.103013,0.320956,0.217407
2,0.095079,0.308348,0.207433,0.103060,0.321029,0.213035
3,0.091729,0.302869,0.200908,0.103678,0.321990,0.213076
4,0.088833,0.298049,0.195237,0.104387,0.323089,0.218133


In [ ]:
num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

#5 layers
hidden = keras.layers.Dense(16, activation='relu')(concat)
hidden = keras.layers.Dense(16, activation='relu')(hidden)
hidden = keras.layers.Dense(16, activation='relu')(hidden)
hidden = keras.layers.Dense(16, activation='relu')(hidden)
hidden = keras.layers.Dense(16, activation='relu')(hidden)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=5, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_layers_5 = pd.DataFrame(model.history.history)
results_layers_5

Epoch 1/5
17868/17868 [==============================] - 55s 3ms/step - loss: 0.1083 - RMSE: 0.3292 - MAE: 0.2330 - val_loss: 0.1043 - val_RMSE: 0.3229 - val_MAE: 0.2255
Epoch 2/5
17868/17868 [==============================] - 49s 3ms/step - loss: 0.0992 - RMSE: 0.3149 - MAE: 0.2154 - val_loss: 0.1029 - val_RMSE: 0.3208 - val_MAE: 0.2103
Epoch 3/5
17868/17868 [==============================] - 48s 3ms/step - loss: 0.0948 - RMSE: 0.3079 - MAE: 0.2068 - val_loss: 0.1032 - val_RMSE: 0.3212 - val_MAE: 0.2155
Epoch 4/5
17868/17868 [==============================] - 49s 3ms/step - loss: 0.0915 - RMSE: 0.3025 - MAE: 0.2004 - val_loss: 0.1039 - val_RMSE: 0.3223 - val_MAE: 0.2193
Epoch 5/5
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0887 - RMSE: 0.2979 - MAE: 0.1950 - val_loss: 0.1042 - val_RMSE: 0.3228 - val_MAE: 0.2136


,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108346,0.329159,0.232976,0.104293,0.322944,0.225530
1,0.099186,0.314938,0.215394,0.102885,0.320756,0.210299
2,0.094782,0.307867,0.206791,0.103163,0.321190,0.215539
3,0.091480,0.302456,0.200439,0.103862,0.322276,0.219304
4,0.088723,0.297864,0.195004,0.104208,0.322812,0.213557


In [ ]:
# Embed implicit Features indivisually

# Embed implicit Features indivisually

# Embed implicit Features indivisually

# Embed implicit Features indivisually

# Embed implicit Features indivisually




# A no implicit features
# B embed isPopular
# C embed isActive
# D embed preferred_program_genre
# E embed all implicit features

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df = pd.read_csv('df.csv')

user_df = train[['user_id_maped']]
movie_df = train[['movie_id','program_class', 'program_genre', 'hd']]

test_user_df = test[['user_id_maped',]]
test_movie_df = test[['movie_id','program_class', 'program_genre', 'hd']]

ohe_user = OneHotEncoder(handle_unknown='ignore',drop='first', min_frequency=5)
user_df = ohe_user.fit_transform(user_df)
test_user_df = ohe_user.transform(test_user_df)

ohe_movie = OneHotEncoder(handle_unknown='ignore',drop='first')
movie_df = ohe_movie.fit_transform(movie_df)
test_movie_df = ohe_movie.transform(test_movie_df)

num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

hidden = keras.layers.Dense(16, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=10, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_A = pd.DataFrame(model.history.history)
results_A


Epoch 1/10


2023-05-23 17:27:28.412456: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 53s 3ms/step - loss: 0.1089 - RMSE: 0.3301 - MAE: 0.2346 - val_loss: 0.1048 - val_RMSE: 0.3237 - val_MAE: 0.2286
Epoch 2/10
17868/17868 [==============================] - 47s 3ms/step - loss: 0.1002 - RMSE: 0.3166 - MAE: 0.2204 - val_loss: 0.1036 - val_RMSE: 0.3219 - val_MAE: 0.2240
Epoch 3/10
17868/17868 [==============================] - 47s 3ms/step - loss: 0.0973 - RMSE: 0.3119 - MAE: 0.2153 - val_loss: 0.1034 - val_RMSE: 0.3216 - val_MAE: 0.2247
Epoch 4/10
17868/17868 [==============================] - 47s 3ms/step - loss: 0.0952 - RMSE: 0.3085 - MAE: 0.2115 - val_loss: 0.1036 - val_RMSE: 0.3218 - val_MAE: 0.2205
Epoch 5/10
17868/17868 [==============================] - 48s 3ms/step - loss: 0.0935 - RMSE: 0.3058 - MAE: 0.2083 - val_loss: 0.1034 - val_RMSE: 0.3216 - val_MAE: 0.2238
Epoch 6/10
17868/17868 [==============================] - 48s 3ms/step - loss: 0.0921 - RMSE: 0.3034 - MAE: 0.2058 - val_loss: 0.1034 - val_RMSE: 0.3215 - v

,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108944,0.330067,0.234552,0.104753,0.323656,0.228553
1,0.100217,0.316571,0.220374,0.103626,0.321910,0.224023
2,0.097251,0.311851,0.215285,0.103428,0.321602,0.224728
3,0.095151,0.308465,0.211506,0.103555,0.321800,0.220485
4,0.093496,0.305771,0.208322,0.103437,0.321616,0.223785
5,0.092058,0.303410,0.205833,0.103390,0.321542,0.220567
6,0.090874,0.301453,0.203668,0.103975,0.322451,0.221549
7,0.089844,0.299740,0.201703,0.104763,0.323671,0.224947
8,0.088934,0.298218,0.200087,0.104294,0.322946,0.223761
9,0.088149,0.296900,0.198615,0.104852,0.323809,0.220385


In [ ]:
user_df = train[['user_id_maped']]
movie_df = train[['movie_id','program_class', 'program_genre', 'hd','isPopular']]

test_user_df = test[['user_id_maped',]]
test_movie_df = test[['movie_id','program_class', 'program_genre', 'hd', 'isPopular']]

ohe_user = OneHotEncoder(handle_unknown='ignore',drop='first', min_frequency=5)
user_df = ohe_user.fit_transform(user_df)
test_user_df = ohe_user.transform(test_user_df)

ohe_movie = OneHotEncoder(handle_unknown='ignore',drop='first')
movie_df = ohe_movie.fit_transform(movie_df)
test_movie_df = ohe_movie.transform(test_movie_df)

num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

hidden = keras.layers.Dense(16, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=10, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_B = pd.DataFrame(model.history.history)
results_B


Epoch 1/10
17868/17868 [==============================] - 51s 3ms/step - loss: 0.1095 - RMSE: 0.3309 - MAE: 0.2351 - val_loss: 0.1051 - val_RMSE: 0.3242 - val_MAE: 0.2306
Epoch 2/10
17868/17868 [==============================] - 45s 3ms/step - loss: 0.1009 - RMSE: 0.3177 - MAE: 0.2199 - val_loss: 0.1040 - val_RMSE: 0.3225 - val_MAE: 0.2267
Epoch 3/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0980 - RMSE: 0.3131 - MAE: 0.2156 - val_loss: 0.1036 - val_RMSE: 0.3219 - val_MAE: 0.2214
Epoch 4/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0962 - RMSE: 0.3101 - MAE: 0.2126 - val_loss: 0.1038 - val_RMSE: 0.3222 - val_MAE: 0.2271
Epoch 5/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0946 - RMSE: 0.3075 - MAE: 0.2099 - val_loss: 0.1035 - val_RMSE: 0.3218 - val_MAE: 0.2251
Epoch 6/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0931 - RMSE: 0.3051 - MAE: 0.2074 - val_loss: 0.1043 - val_RMSE:

,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.109501,0.330909,0.235116,0.105122,0.324225,0.230620
1,0.100943,0.317715,0.219910,0.104004,0.322497,0.226742
2,0.098038,0.313111,0.215561,0.103596,0.321864,0.221377
3,0.096170,0.310114,0.212616,0.103803,0.322185,0.227096
4,0.094559,0.307505,0.209929,0.103534,0.321767,0.225129
5,0.093105,0.305130,0.207428,0.104340,0.323017,0.227499
6,0.091905,0.303158,0.205368,0.104622,0.323454,0.221617
7,0.090792,0.301317,0.203238,0.104289,0.322939,0.220593
8,0.089840,0.299733,0.201554,0.104627,0.323461,0.218334
9,0.089013,0.298350,0.200013,0.104785,0.323705,0.221011


In [ ]:
user_df = train[['user_id_maped', 'isActive']]
movie_df = train[['movie_id','program_class', 'program_genre', 'hd']]

test_user_df = test[['user_id_maped', 'isActive']]
test_movie_df = test[['movie_id','program_class', 'program_genre', 'hd']]

ohe_user = OneHotEncoder(handle_unknown='ignore',drop='first', min_frequency=5)
user_df = ohe_user.fit_transform(user_df)
test_user_df = ohe_user.transform(test_user_df)

ohe_movie = OneHotEncoder(handle_unknown='ignore',drop='first')
movie_df = ohe_movie.fit_transform(movie_df)
test_movie_df = ohe_movie.transform(test_movie_df)

num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

hidden = keras.layers.Dense(16, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=10, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_C = pd.DataFrame(model.history.history)
results_C

Epoch 1/10


2023-05-23 18:22:37.276541: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 54s 3ms/step - loss: 0.1086 - RMSE: 0.3295 - MAE: 0.2343 - val_loss: 0.1044 - val_RMSE: 0.3231 - val_MAE: 0.2268
Epoch 2/10
17868/17868 [==============================] - 47s 3ms/step - loss: 0.0999 - RMSE: 0.3161 - MAE: 0.2204 - val_loss: 0.1035 - val_RMSE: 0.3217 - val_MAE: 0.2290
Epoch 3/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0970 - RMSE: 0.3114 - MAE: 0.2157 - val_loss: 0.1036 - val_RMSE: 0.3219 - val_MAE: 0.2219
Epoch 4/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0950 - RMSE: 0.3083 - MAE: 0.2123 - val_loss: 0.1034 - val_RMSE: 0.3216 - val_MAE: 0.2219
Epoch 5/10
17868/17868 [==============================] - 47s 3ms/step - loss: 0.0935 - RMSE: 0.3057 - MAE: 0.2098 - val_loss: 0.1038 - val_RMSE: 0.3222 - val_MAE: 0.2215
Epoch 6/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0922 - RMSE: 0.3036 - MAE: 0.2076 - val_loss: 0.1038 - val_RMSE: 0.3222 - v

,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108551,0.329471,0.234311,0.104402,0.323113,0.226812
1,0.099898,0.316066,0.220396,0.103512,0.321733,0.228998
2,0.096979,0.311414,0.215677,0.103642,0.321935,0.221879
3,0.095024,0.308259,0.212322,0.103444,0.321628,0.221924
4,0.093452,0.305699,0.209776,0.103785,0.322156,0.221483
5,0.092169,0.303594,0.207587,0.103836,0.322236,0.221843
6,0.091020,0.301696,0.205481,0.104538,0.323324,0.232525
7,0.090055,0.300092,0.203774,0.104548,0.323338,0.223820
8,0.089139,0.298562,0.202121,0.105057,0.324124,0.222809
9,0.088351,0.297239,0.200647,0.105411,0.324671,0.225084


In [ ]:
user_df = train[['user_id_maped', 'preferred_program_genre']]
movie_df = train[['movie_id','program_class', 'program_genre', 'hd']]

test_user_df = test[['user_id_maped', 'preferred_program_genre']]
test_movie_df = test[['movie_id','program_class', 'program_genre', 'hd']]

ohe_user = OneHotEncoder(handle_unknown='ignore',drop='first', min_frequency=5)
user_df = ohe_user.fit_transform(user_df)
test_user_df = ohe_user.transform(test_user_df)

ohe_movie = OneHotEncoder(handle_unknown='ignore',drop='first')
movie_df = ohe_movie.fit_transform(movie_df)
test_movie_df = ohe_movie.transform(test_movie_df)

num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

hidden = keras.layers.Dense(16, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=10, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_D = pd.DataFrame(model.history.history)
results_D

Epoch 1/10
17868/17868 [==============================] - 52s 3ms/step - loss: 0.1091 - RMSE: 0.3302 - MAE: 0.2353 - val_loss: 0.1052 - val_RMSE: 0.3243 - val_MAE: 0.2226
Epoch 2/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.1001 - RMSE: 0.3164 - MAE: 0.2207 - val_loss: 0.1035 - val_RMSE: 0.3217 - val_MAE: 0.2269
Epoch 3/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0970 - RMSE: 0.3115 - MAE: 0.2155 - val_loss: 0.1030 - val_RMSE: 0.3209 - val_MAE: 0.2241
Epoch 4/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0948 - RMSE: 0.3080 - MAE: 0.2121 - val_loss: 0.1029 - val_RMSE: 0.3208 - val_MAE: 0.2250
Epoch 5/10
17868/17868 [==============================] - 46s 3ms/step - loss: 0.0931 - RMSE: 0.3052 - MAE: 0.2091 - val_loss: 0.1030 - val_RMSE: 0.3209 - val_MAE: 0.2231
Epoch 6/10
17868/17868 [==============================] - 44s 2ms/step - loss: 0.0917 - RMSE: 0.3028 - MAE: 0.2066 - val_loss: 0.1033 - val_RMSE:

,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.109065,0.330250,0.235290,0.105153,0.324274,0.222621
1,0.100139,0.316448,0.220695,0.103517,0.321740,0.226936
2,0.097013,0.311469,0.215528,0.102999,0.320935,0.224054
3,0.094834,0.307951,0.212073,0.102905,0.320788,0.224962
4,0.093125,0.305164,0.209139,0.103006,0.320946,0.223139
5,0.091709,0.302835,0.206648,0.103254,0.321332,0.224410
6,0.090439,0.300730,0.204458,0.103654,0.321954,0.224872
7,0.089378,0.298962,0.202495,0.104104,0.322652,0.218064
8,0.088419,0.297353,0.200757,0.104332,0.323005,0.223173
9,0.087614,0.295997,0.199228,0.104859,0.323819,0.223504


In [ ]:
user_df = train[['user_id_maped','isActive', 'preferred_program_genre']]
movie_df = train[['movie_id','program_class', 'program_genre', 'hd', 'isPopular']]

test_user_df = test[['user_id_maped','isActive', 'preferred_program_genre']]
test_movie_df = test[['movie_id','program_class', 'program_genre', 'hd', 'isPopular']]

ohe_user = OneHotEncoder(handle_unknown='ignore',drop='first', min_frequency=5)
user_df = ohe_user.fit_transform(user_df)
test_user_df = ohe_user.transform(test_user_df)

ohe_movie = OneHotEncoder(handle_unknown='ignore',drop='first')
movie_df = ohe_movie.fit_transform(movie_df)
test_movie_df = ohe_movie.transform(test_movie_df)


num_user_features = user_df.shape[1]
num_item_features = movie_df.shape[1]

user_input = keras.Input(shape=(num_user_features,))
item_input = keras.Input(shape=(num_item_features,))

concat = keras.layers.Concatenate()([user_input, item_input])

hidden = keras.layers.Dense(16, activation='relu')(concat)

output = keras.layers.Dense(1)(hidden)

model = keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError(name='RMSE'), tf.keras.metrics.MeanAbsoluteError(name='MAE')])

model.fit([user_df.toarray(), movie_df.toarray()], train['rating'].values, epochs=10, validation_data=([test_user_df.toarray(), test_movie_df.toarray()], test['rating'].values))

results_E = pd.DataFrame(model.history.history)
results_E

Epoch 1/10


2023-05-23 18:43:22.329948: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17868/17868 [==============================] - 54s 3ms/step - loss: 0.1086 - RMSE: 0.3296 - MAE: 0.2347 - val_loss: 0.1044 - val_RMSE: 0.3231 - val_MAE: 0.2284
Epoch 2/10
17868/17868 [==============================] - 49s 3ms/step - loss: 0.0998 - RMSE: 0.3159 - MAE: 0.2205 - val_loss: 0.1032 - val_RMSE: 0.3212 - val_MAE: 0.2247
Epoch 3/10
17868/17868 [==============================] - 47s 3ms/step - loss: 0.0965 - RMSE: 0.3107 - MAE: 0.2154 - val_loss: 0.1032 - val_RMSE: 0.3212 - val_MAE: 0.2283
Epoch 4/10
17868/17868 [==============================] - 50s 3ms/step - loss: 0.0943 - RMSE: 0.3071 - MAE: 0.2115 - val_loss: 0.1030 - val_RMSE: 0.3210 - val_MAE: 0.2224
Epoch 5/10
17868/17868 [==============================] - 47s 3ms/step - loss: 0.0926 - RMSE: 0.3043 - MAE: 0.2085 - val_loss: 0.1031 - val_RMSE: 0.3211 - val_MAE: 0.2225
Epoch 6/10
17868/17868 [==============================] - 48s 3ms/step - loss: 0.0912 - RMSE: 0.3019 - MAE: 0.2059 - val_loss: 0.1035 - val_RMSE: 0.3217 - v

,loss,RMSE,MAE,val_loss,val_RMSE,val_MAE
0,0.108635,0.329598,0.234666,0.104419,0.323139,0.228355
1,0.099789,0.315893,0.220475,0.103182,0.321220,0.224670
2,0.096542,0.310713,0.215371,0.103191,0.321234,0.228312
3,0.094334,0.307138,0.211500,0.103024,0.320974,0.222351
4,0.092594,0.304292,0.208469,0.103122,0.321127,0.222486
5,0.091153,0.301916,0.205924,0.103465,0.321659,0.223544
6,0.089962,0.299936,0.203726,0.103363,0.321501,0.221996
7,0.088902,0.298165,0.201857,0.103972,0.322446,0.218790
8,0.087972,0.296600,0.200187,0.104508,0.323278,0.225226
9,0.087144,0.295202,0.198682,0.104903,0.323888,0.224609
